# 05 · Prédiction du nombre de médailles par pays/édition

Modélisation de type régression pour estimer le nombre de médailles (total ou par type) qu'un pays remportera lors d'une édition donnée.

## Feuille de route
- Charger les agrégations `country_year_summary.csv`.
- Préparer les features (historique de médailles, nombre d'athlètes, rang moyen).
- Définir plusieurs cibles : total, or, argent, bronze.
- Entraîner des modèles de régression supervisée (baseline : RandomForestRegressor).
- Evaluer via MAE/RMSE et sélectionner le meilleur modèle pour chaque cible.
- Appliquer une projection sur Paris 2024 (si données disponibles) ou France Top 25.
- Sauvegarder les prédictions pour la webapp et les rapports.

In [6]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import joblib

sns.set_theme(style='whitegrid', context='talk')
np.random.seed(42)

In [2]:
BASE_DIR = Path('..').resolve()
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = BASE_DIR / 'models'
REPORTS_DIR = BASE_DIR / 'reports'
FIG_DIR = REPORTS_DIR / 'figures'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

summary_path = PROCESSED_DIR / 'country_year_summary.csv'
if not summary_path.exists():
    raise FileNotFoundError('country_year_summary.csv introuvable. Exécuter 02_preprocessing.ipynb avant ce notebook.')

summary_df = pd.read_csv(summary_path)
summary_df.head(3)

,country_name,slug_game,medals_total,athletes_unique,avg_rank
0,Afghanistan,athens-2004,0,2,19.5
1,Afghanistan,atlanta-1996,0,1,111.0
2,Afghanistan,beijing-2008,1,2,5.0


## Cibles et features
On suppose que `summary_df` contient pour chaque pays/édition : `medals_total`, `athletes_unique`, `avg_rank`. Ajouter `gold`, `silver`, `bronze` si disponibles (sinon fusionner au préalable).

In [3]:
target_cols = ['medals_total']  # étendre à ['medals_total', 'gold', 'silver', 'bronze'] si colonnes présentes
feature_cols = [c for c in summary_df.columns if c not in target_cols + ['country_name', 'slug_game']]

summary_df[feature_cols].head()

,athletes_unique,avg_rank
0,2,19.5
1,1,111.0
2,2,5.0
3,0,5.0
4,0,12.0


## Pipeline de régression
Structure similaire à la classification mais ciblée sur des variables numériques + catégorielles (pays, saison, etc.).

In [4]:
X = summary_df[feature_cols]

## Boucle de modèles par cible
Pour chaque cible, on entraîne un modèle et on stocke les scores.

In [7]:
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

models = {
    'linear_regression': LinearRegression(),
    'random_forest': RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
}

results_records = []
trained_models = {}

In [9]:
for target in target_cols:
    y = summary_df[target].fillna(0)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    for model_name, model in models.items():
        pipe = Pipeline([
            ('preprocessor', preprocessor),
            ('model', clone(model))
        ])
        pipe.fit(X_train, y_train)
        preds = pipe.predict(X_test)
        mae = mean_absolute_error(y_test, preds)
        rmse = float(np.sqrt(mean_squared_error(y_test, preds)))
        cv_pipe = Pipeline([
            ('preprocessor', preprocessor),
            ('model', clone(model))
        ])
        cv_scores = cross_val_score(
            cv_pipe,
            X_train,
            y_train,
            cv=5,
            scoring='neg_mean_absolute_error'
        )

        results_records.append(
            {
                'target': target,
                'model': model_name,
                'MAE': mae,
                'RMSE': rmse,
                'CV_MAE': -cv_scores.mean()
            }
        )
        trained_models[(target, model_name)] = pipe

## Sélection du meilleur modèle
On choisit l'algorithme le plus performant (par exemple RandomForest si MAE minimal).

In [10]:
results_df = (
    pd.DataFrame(results_records)
    .set_index(['target', 'model'])
    .sort_values('MAE')
)
results_df

MAE       RMSE    CV_MAE
target       model                                           
medals_total random_forest      6.098995  17.667503  6.027480
             linear_regression  7.935099  14.936227  8.139373

## Re-entraînement sur tout le dataset et sauvegarde
On entraîne le meilleur modèle identifié sur l'ensemble des données et on stocke l'objet ainsi que les prédictions.

In [11]:
best_target, best_model_name = results_df['MAE'].idxmin()
best_model = models[best_model_name]
best_target, best_model_name

('medals_total', 'random_forest')

In [12]:
final_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', clone(best_model))
])
final_pipe.fit(X, summary_df[best_target].fillna(0))
preds_all = final_pipe.predict(X)

## Focus sur la France & projection Paris 2024
Exemple d'extraction des prédictions pour un pays donné et l'édition 2024.

In [13]:
prediction_df = summary_df[['country_name', 'slug_game']].copy()

In [14]:
prediction_df[f'predicted_{best_target}'] = preds_all
prediction_df.head()

,country_name,slug_game,predicted_medals_total
0,Afghanistan,athens-2004,0.983333
1,Afghanistan,atlanta-1996,0.000000
2,Afghanistan,beijing-2008,1.081667
3,Afghanistan,berlin-1936,3.171500
4,Afghanistan,london-1948,4.980000


In [15]:
france_predictions = prediction_df[prediction_df['country_name'] == 'France']
france_predictions.sort_values('slug_game').tail(5)

,country_name,slug_game,predicted_medals_total
1314,France,sydney-2000,118.513333
1315,France,tokyo-1964,66.333333
1316,France,tokyo-2020,276.050000
1317,France,turin-2006,30.350000
1318,France,vancouver-2010,40.763333


## Sauvegardes
- Modèle entraîné (`joblib`).
- Tableau des prédictions (`csv`).
- Résumé des performances (`csv`).

In [16]:
model_path = MODELS_DIR / f'reg_{best_target}_{best_model_name}.joblib'

In [17]:
joblib.dump(final_pipe, model_path)
prediction_path = REPORTS_DIR / 'medal_predictions.csv'
prediction_df.to_csv(prediction_path, index=False)
scores_path = REPORTS_DIR / 'medal_regression_scores.csv'
results_df.to_csv(scores_path)
model_path, prediction_path, scores_path

(WindowsPath('C:/Users/hicha/Documents/Projets/Olympique_JO/models/reg_medals_total_random_forest.joblib'),
 WindowsPath('C:/Users/hicha/Documents/Projets/Olympique_JO/reports/medal_predictions.csv'),
 WindowsPath('C:/Users/hicha/Documents/Projets/Olympique_JO/reports/medal_regression_scores.csv'))

## Idées d'amélioration
- Ajouter des variables temporelles (tendance sur les 3 dernières éditions, ratio médailles/athlètes).
- Intégrer des données externes (PIB, population, dépenses sportives).
- Tester d'autres algorithmes (XGBoost, LightGBM) ou des approches multivariées (multi-output regressor).
- Construire un scénario Paris 2024 en injectant des hypothèses sur les athlètes engagés.